In [3]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [5]:
path = "../data/raw/household_power_consumption.txt"

df = pd.read_csv(
    path,
    sep=";",
    na_values="?",
    low_memory=False
)

df["datetime"] = pd.to_datetime(
    df["Date"] + " " + df["Time"],
    format="%d/%m/%Y %H:%M:%S"
)

df = df.set_index("datetime")

df["Global_active_power"] = pd.to_numeric(
    df["Global_active_power"], errors="coerce"
)

df = df[["Global_active_power"]].dropna()

series = df["Global_active_power"].resample("1H").mean().dropna()

series.head()

C:\Users\kudic\AppData\Local\Temp\ipykernel_14204\2455281967.py:23: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  series = df["Global_active_power"].resample("1H").mean().dropna()


datetime
2006-12-16 17:00:00    4.222889
2006-12-16 18:00:00    3.632200
2006-12-16 19:00:00    3.400233
2006-12-16 20:00:00    3.268567
2006-12-16 21:00:00    3.056467
Name: Global_active_power, dtype: float64

In [6]:
train_size = int(len(series) * 0.8)

train = series.iloc[:train_size]
test = series.iloc[train_size:]

len(train), len(test)

(27334, 6834)

In [7]:
model = ARIMA(train, order=(2, 1, 2))
fitted = model.fit()

forecast = fitted.forecast(steps=len(test))
forecast.head()

C:\Users\kudic\Projects\csan-energy-forecasting\.venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\kudic\Projects\csan-energy-forecasting\.venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\kudic\Projects\csan-energy-forecasting\.venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\kudic\Projects\csan-energy-forecasting\.venv\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA

27334    0.773159
27335    1.115968
27336    1.376710
27337    1.551854
27338    1.652785
Name: predicted_mean, dtype: float64

In [9]:
mae = mean_absolute_error(test, forecast)
mse = mean_squared_error(test, forecast)
rmse = np.sqrt(mse)

mae, rmse

(0.8113837921704088, np.float64(0.9349483354101888))